### Not MNIST Data Classifiaction
Given a <a href='http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html'> Data set </a> of not MNIST Data
which have 10 classes of letter A-J our Job os to classify the Images


Importing Necessary Libray

In [1]:
from __future__ import print_function
import os
import glob
import pickle
from typing import List, Dict, Tuple
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.preprocessing.image as tfpreprocess
%matplotlib inline

Setting Up the constants train folder and test folder can be easily obtain by extracting the notMNIST dataset
from the link provided at the top

In [2]:
# setting constants used
NUM_CLASSES = 10 # total number of classes
IMAGE_SIZE = 28  # Pixel width and height.
PIXEL_DEPTH = 255.0  # Number of levels per pixel
PATH_TRAIN = []
SEED = 42 # for permutation in data shuffuling
PATH_TEST = []
TRAIN_FOLDER = 'notMNIST_large' # Folder that contain all the alphabets image folder wise 
TEST_FOLDER = 'notMNIST_small' # Folder that contain all the alphabets image folder wise 

check number of sub folder is equal to NUM_CLASS (number of subfolder in train folder)

In [3]:
sub_directory = list(os.listdir(TRAIN_FOLDER))
for sub in sub_directory:
    merge_path = os.path.join(TRAIN_FOLDER, sub)
    if not os.path.isfile(merge_path):
        PATH_TRAIN.append(merge_path)   # add mapping  merge path to class Name tuple
assert len(PATH_TRAIN) == NUM_CLASSES

load all the sub path of test data note it may may not be equal to the NUM_CLASS
since it is not mendatory all classes data are present in test samples

In [4]:
sub_directory = list(os.listdir(TEST_FOLDER))
for sub in sub_directory:
    merge_path = os.path.join(TEST_FOLDER, sub)
    if not os.path.isfile(merge_path):
        PATH_TEST.append(merge_path)  # add mapping  merge path to class Name tuple

object form of an image containing Information About Images and Its vector

In [5]:
def get_pickel_object(image_path: str, extension: str, data: object,
                                  width: int, height: int, label: str)->object:
    """
    Get Pickle object which contain all relative Information of Image
    
    parameter
    ----------
    
    image_path: str
        path of the Image from where It has been loaded
    
    extension: str
        extension of Image .png, .jpg, e.t.c
        
    data: str
        vector if (IMAGE_SIZE, IMAGE_SIZE) number representing grayscale pixels value
    
    width: int
        width of the Image
    
    height: int
        height of the Image
    
    label; string
        Label of the Image which It belongs
    
    """
    return {
            'imagePath': image_path,
            'extension': extension,
            'width': width,
            'height': height,
            'data': data,
            'label': label
           }
# End

Generate Augumented Image From random sample Image List of Data with Random Sample behaviour ans Also Save
The Augumented Image If save_augumented_image set it to true in the same directory of image sample

In [6]:
def image_augumentation_generator(data: List[object], max_count: int, save_augumented_image=False)->List[object]:
    """
    Generate augumented Image of till data length not equal to max_count passes from paramter
    
    paramter
    -----------
    
    data: List[object]
        list of objects (containing property {'imagePath' ,'extension', 'width', 'height', 'data', 'label'})
        
    max_count: int
        maximum number of augumented image can be created (max_count - len(data) augumented image data)
    
    save_augumented_image: bool
        If true then augumented Image is saved to Disk In the same directry from which random sample from data list image is taken
        saved Image format is  [imagelabel]_[imagename]_augumented_image_[number].[extension]
    
    returns
    ----------
        List[object]
        object list of augumented Images
    """
    ############################  Image Generator ###########################################
    image_gen = [tfpreprocess.ImageDataGenerator(zoom_range=0.2),
                 tfpreprocess.ImageDataGenerator(horizontal_flip=True, vertical_flip=True),
                 tfpreprocess.ImageDataGenerator(zoom_range=0.5),
                 tfpreprocess.ImageDataGenerator(vertical_flip=True),
                 tfpreprocess.ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.1),
                 tfpreprocess.ImageDataGenerator(rotation_range=45),
                 tfpreprocess.ImageDataGenerator(rotation_range=90),
                 tfpreprocess.ImageDataGenerator(width_shift_range=0.3, height_shift_range=0.1, rotation_range=90, vertical_flip=True),
                 tfpreprocess.ImageDataGenerator(width_shift_range=0.23, height_shift_range=0.01, rotation_range=90),
                 tfpreprocess.ImageDataGenerator(channel_shift_range=0.9),
                 tfpreprocess.ImageDataGenerator(channel_shift_range=0.9, rotation_range=90)
                ]
    
    n, n_igen = len(data), len(image_gen)
    diff = max_count - n
    ls = []
    
    for i in range(1, diff+1):
        img_gen_idx = np.random.randint(n_igen)
        image_selected_idx = np.random.randint(n)
        img_gen_selected = image_gen[img_gen_idx]
        image = data[image_selected_idx]
        datagen = img_gen_selected.flow(image['data'].reshape(1, 1, IMAGE_SIZE, IMAGE_SIZE), batch_size = 1)
        new_image_v = datagen.next()[0].astype('uint8').reshape(IMAGE_SIZE, IMAGE_SIZE)
        img_path_split =  os.path.split(image['imagePath'])
        img_name_split = os.path.splitext(img_path_split[1])
        
        if save_augumented_image:
            image_name = '{0}_{1}_augumented_image_{2}{3}'.format(image['label'], img_name_split[0], str(i), img_name_split[1])
            directory = img_path_split[0]
            image_path = os.path.join(directory, image_name)
            image_save = tfpreprocess.array_to_img(new_image_v.reshape(28, 28, 1))
            image_save.save(image_path)
        else:
            image_path = ''
        obj = get_pickel_object(image_path, image['extension'], new_image_v,
                                    IMAGE_SIZE, IMAGE_SIZE, image['label'])
        ls.append(obj)
    
    return ls
# End

function to retrieve data from sub folder i.e. notMNISTlarge/A or  notMNISTlarge/B e.t.c
dataset create List of object obtained from get_pickel_object() function and return it

In [8]:
def load_image_from_path(image_folder: str, label: str)->List[object]:
    """
    utility function to Load Image From the path provided
    
    paramter
    ---------
    
    image_folder: str
        image folder path containing Images of labels provided in parameter
        
    label: str
        label Name of all Images under given image_folder
    
    returns
    --------
        List[object]
        List of image object defined in function get_pickel_object
    """
    global IMAGE_SIZE
   
    folder_list = list(os.path.split(image_folder))
    print('Initialting '+label+' Data loading...')
    image_files = os.listdir(image_folder)
    ls = []
    progressbar = '============================='
    PROGRESS_BAR_SIZE = 500
    ctr, n = 1, len(image_files)
    for imagename in image_files:

        image_path = os.path.join(*(folder_list+[imagename]))

        if not os.path.isfile(image_path):
            continue
        try:
            extension = os.path.splitext(imagename)[1]
            image = tfpreprocess.load_img(image_path, color_mode="grayscale",
                                        target_size=(IMAGE_SIZE, IMAGE_SIZE))

            data = tfpreprocess.img_to_array(image).reshape(IMAGE_SIZE, IMAGE_SIZE).astype("float32")
            obj = get_pickel_object(image_path, extension, data, IMAGE_SIZE, IMAGE_SIZE, label)
            ls.append(obj)
        except IOError as e:
            print('skipping file IO Error occured '+str(e))
        
        if ctr%PROGRESS_BAR_SIZE == 0:
            info = progressbar+' {0}/{1} '.format(ctr, n) + progressbar
            print(info)
        
        ctr += 1
    
    if ctr <= n:
        info = progressbar+' {0}/{1} '.format(ctr, n) + progressbar
        print(info)
    print('\n{0} data loaded\n'.format(label))
    return ls
# End

Main function to load the data If pickle file found then load from pickle file directly if
force parameter is False else override existing pickle file and return dictionary Label to dataset mapping

In [9]:
def load_dataset_from_disk(pathlist: List[str], picklefilefolder: str, force=False)-> Tuple[Dict[str,Tuple[str, List[object]]], int]:
    """
    Load all Images from given path List
    
    parameter
    ---------
        
    pathlist: List[str]
        list of paths of folder from which images need to loaded
    
    picklefilefolder: str
        pickle folder path If pickled file found then load from pickle file it will save time
    
    force: bool
        If set to True then Image load from folders not from pickle file
    
    returns
    ----------
        Tuple[Dict[str List[object]], int]
        Tuple of dictionary of label to list of image object mapping , maximum number of Images of particular Label
    """
    res = {}
    max_size = 0
    for path in pathlist:
    
        label = os.path.split(path)[-1]
        pickefilepath = os.path.join(picklefilefolder, label+'.pickle')
        
        if os.path.exists(pickefilepath) and not force:
            print(pickefilepath+' is already exist loading from pickle file....')
            
            with open(pickefilepath, 'rb') as pf:
                resultdataset = pickle.load(pf) 
                res[label] = (pickefilepath, resultdataset)
        else:
            
            resultdataset = load_image_from_path(path, label)
            res[label] = (pickefilepath, resultdataset)
        
        max_size = max(max_size, len(resultdataset))
    
    return (res, max_size)
# End

save data into pickle file which having the size specified in parameter If
pickle file already present then override it provided override to True

In [10]:
def save_dataset_into_pickle_file(dataset: List[object], picklefilepath: str, datasize: int,
                                              data_size_modification=False, override=False)->None:
    """
    save data into pickle file
    
    parameter
    ---------
    
    dataset: List[object]
        list of image objects
    
    picklefilepath: str
        pickle file with path where pickle file need to save
    
    datasize: int
        data size of image object list need to be saved in pickle file
    
    data_size_modification: bool
        If true then dataset modification takes place If list of image object
        less then given datasize augumented image data will be added, if more
        data found then truncated the data and saved into pickle file
    
    override: bool
        if set to true then if pickle file already existing then override it
    
    return
    -------
        None
        
    """
    try:
        n = len(dataset)
        if data_size_modification:
            if n < datasize:
                print('\nless Image found addeing more augumented Data...')
                # make last argument true If you want to save Image in same directory
                aug_list = image_augumentation_generator(dataset, datasize, False)
                dataset.extend(aug_list)
            elif n > datasize:
                print('more Image found truncationg Data...')
                dataset = dataset[:datasize]
            else:
                print('Data size is equal to Expected size..')
        else:
            print('No modification of Data take place.')
        
        if os.path.exists(picklefilepath) and not override:
            print('File {0} already exists skipping...'.format(picklefilepath))
            return
        with open(picklefilepath, 'wb') as pf:
            pickle.dump(dataset, pf, protocol=pickle.HIGHEST_PROTOCOL)
        print('Data saved successfully to {0} path data size = {1}'.format(picklefilepath, datasize))
    except:
        raise
#End

return dataset dictionary label to dataset mapping with fixed size if size is specified then
If override is specified it wite pickle file to disk if size if not specified then map the
data as it is

In [11]:
def get_dataset(datadict:Dict[str, Tuple[str, List[object]]], size: int,
                data_size_modification=False, override=False)->Dict[str, List[object]]:
    """
     Get Data of specified size in parameter
     paramter
     --------
     datadict: Dict[str, Tuple[str, List[object]]]
         data dictionary obtained from function load_dataset_from_disk  contain key as label
         and value as tuple of picklepath and list of image object
        
    size: int
        size of each laebl data needed only effective if data_size_modification is true
    
    data_size_modification: bool
        if set to true then only list of Image object modification appending augumented data
        or truncating data is allowed other wise skipping it
    
    override: bool
         If true then pickle file is overriden with new One
        
    return
    ---------
        Dict[str, List[object]]
        dictionary with label to List of imae objects
    
    """
    print('Max size of Data {0}'.format(size))
    datasets = {}
    for label, data in datadict.items(): # key is lebel and data is tuple of pickle file path and dataset
        try:
            picklepath, dataset = data[0], data[1]
            n = len(dataset)
            print('Label {0} Data size {1}'.format(label, n))
            save_dataset_into_pickle_file(dataset, picklepath, size, data_size_modification, override)
            datasets[label] = dataset
        except:
            print(data)
    return datasets
# End

loading train data from disk 

In [12]:
dataset_dict_train, max_train_size  = load_dataset_from_disk(pathlist=PATH_TRAIN,
                                                             picklefilefolder=TRAIN_FOLDER,
                                                             force=False)

notMNIST_large\A.pickle is already exist loading from pickle file....
notMNIST_large\B.pickle is already exist loading from pickle file....
notMNIST_large\C.pickle is already exist loading from pickle file....
notMNIST_large\D.pickle is already exist loading from pickle file....
notMNIST_large\E.pickle is already exist loading from pickle file....
notMNIST_large\F.pickle is already exist loading from pickle file....
notMNIST_large\G.pickle is already exist loading from pickle file....
notMNIST_large\H.pickle is already exist loading from pickle file....
notMNIST_large\I.pickle is already exist loading from pickle file....
notMNIST_large\J.pickle is already exist loading from pickle file....


Loading test data from disk

In [13]:
dataset_dict_test, max_test_size  = load_dataset_from_disk(pathlist=PATH_TEST,
                                                           picklefilefolder=TEST_FOLDER,
                                                           force=False)

notMNIST_small\A.pickle is already exist loading from pickle file....
notMNIST_small\B.pickle is already exist loading from pickle file....
notMNIST_small\C.pickle is already exist loading from pickle file....
notMNIST_small\D.pickle is already exist loading from pickle file....
notMNIST_small\E.pickle is already exist loading from pickle file....
notMNIST_small\F.pickle is already exist loading from pickle file....
notMNIST_small\G.pickle is already exist loading from pickle file....
notMNIST_small\H.pickle is already exist loading from pickle file....
notMNIST_small\I.pickle is already exist loading from pickle file....
notMNIST_small\J.pickle is already exist loading from pickle file....


Save the loaded train data into pickle file if override=True else make modify
the data to equal length if data_size_modification=True is set else return the
data dict label to data object list

In [14]:
train_dict = get_dataset(datadict=dataset_dict_train, size=max_train_size, data_size_modification=True, override=False)

Max size of Data 52912
Label A Data size 52912
Data size is equal to Expected size..
File notMNIST_large\A.pickle already exists skipping...
Label B Data size 52912
Data size is equal to Expected size..
File notMNIST_large\B.pickle already exists skipping...
Label C Data size 52912
Data size is equal to Expected size..
File notMNIST_large\C.pickle already exists skipping...
Label D Data size 52912
Data size is equal to Expected size..
File notMNIST_large\D.pickle already exists skipping...
Label E Data size 52912
Data size is equal to Expected size..
File notMNIST_large\E.pickle already exists skipping...
Label F Data size 52912
Data size is equal to Expected size..
File notMNIST_large\F.pickle already exists skipping...
Label G Data size 52912
Data size is equal to Expected size..
File notMNIST_large\G.pickle already exists skipping...
Label H Data size 52912
Data size is equal to Expected size..
File notMNIST_large\H.pickle already exists skipping...
Label I Data size 52912
Data size

Save the loaded test data into pickle file if override=True else make modify
the data to equal length if data_size_modification=True is set else return the
data dict label to data object list

In [15]:
test_dict = get_dataset(datadict=dataset_dict_test, size=max_test_size, data_size_modification=False, override=False)

Max size of Data 1873
Label A Data size 1872
No modification of Data take place.
File notMNIST_small\A.pickle already exists skipping...
Label B Data size 1873
No modification of Data take place.
File notMNIST_small\B.pickle already exists skipping...
Label C Data size 1873
No modification of Data take place.
File notMNIST_small\C.pickle already exists skipping...
Label D Data size 1873
No modification of Data take place.
File notMNIST_small\D.pickle already exists skipping...
Label E Data size 1873
No modification of Data take place.
File notMNIST_small\E.pickle already exists skipping...
Label F Data size 1872
No modification of Data take place.
File notMNIST_small\F.pickle already exists skipping...
Label G Data size 1872
No modification of Data take place.
File notMNIST_small\G.pickle already exists skipping...
Label H Data size 1872
No modification of Data take place.
File notMNIST_small\H.pickle already exists skipping...
Label I Data size 1872
No modification of Data take place.

retrieve all the class Names and save into list from train_dict

In [16]:
CLASS_LIST = list(train_dict.keys())

In [17]:
def get_class_map_and_inverse_map(class_list: List[str])-> Tuple[Dict[str, int], Dict[int, str]]:
    """
    label encoder and decoder
    paramter
    --------
    
    class_list: List[str]
        list of Labels
    
    return 
    --------
        Tuple[Dict[str, int], Dict[int, str]]
        tuple of (label to integer dictionary, integer to label dictionary)
    """
    class_map = {x: class_list.index(x) for x in class_list}
    class_inv_map = {class_list.index(x): x for x in class_list}
    return (class_map, class_inv_map)
# End

save class Label to integer and Inverse map integer to class Label mapping

In [18]:
CLASS_MAP, CLASS_INV_MAP = get_class_map_and_inverse_map(CLASS_LIST)

Merge all dataset into single one 

In [19]:
def merge_dataset(data_train: Dict[str, List[object]], size: int, label_map: Dict[str, int], validation_size=None):
    """
    merge Multiple Data Into single numpy array
    paramter
    ----------
    
    data_train: Dict[str, List[object]]
        mapping label to data object list
    
    size: int
        size of each label data sample if -1 given take all the data else truncate the rest of the data
    
    label_map: Dict[str, int]
        label Incoded mapping of label to number
    
    validation_size: int  (optional)
        If provided then create that much size validation set from truncated data after size provided in args
    
    returns
    -----------
    Tuple[np.array, np.array, np.array, np.array]
        numpy array of dataset, dataset label, validation set, validation label 
        Ignore validation set if validation_size None provided as it return empty list
    """
    def get_data(x, label):
        lb = np.zeros(len(label_map), dtype=int)
        lb[label_map[x['label']]] = 1
        label.append(lb)
        return x['data']
    # End
    
    datasets, labels, dataset_val, label_val = [], [], [], []
    for label, data in data_train.items():
        n = len(data)
        label = []
        dataset = list(map(lambda x : get_data(x, label), data))
        if size < 0:
            datasets.extend(dataset)
            labels.extend(label)
            continue
        
        if validation_size is not None and (size+validation_size) > n:
            raise Exception('Available data is {0} required Data is {1}'.format(n, size+validation_size))
        
        if validation_size is None and size > n:
            raise Exception('Available data is {0} required Data is {1}'.format(n, size))
        
        datasets.extend(dataset[:size])
        labels.extend(label[:size])
        
        if validation_size is not None:
            dataset_val.extend(dataset[size:size+validation_size])
            label_val.extend(label[size:size+validation_size])
    datasets, labels = np.array(datasets), np.array(labels)
    dataset_val, label_val = np.array(dataset_val), np.array(label_val)
    
    return (datasets, labels, dataset_val, label_val)
# End    

random shuffuling of data 

In [20]:
def randomize(dataset: np.array, labels: np.array)->(np.array, np.array):
    """
    random shuffle of dataset
    
    parameter
    ----------
        dataset: np.array (numpy array)
            dataset neet to shuffle
        
        labels: np.array (numpy array)
            label assosiate with dataset
    return np.array, np.array
        random shuffled dataset and labels
    """
    global SEED
    if dataset.shape[0] != labels.shape[0]:
        raise Exception('labels and data set length mismatch label len:- {0} dataset len:- {1}'.format(dataset.shape[0],labels.shape[0]))
    np.random.seed(SEED)
    permutation = np.random.permutation(labels.shape[0])
    sh_dataset = dataset[permutation, :, :]
    sh_label = labels[permutation, :]
    return sh_dataset, sh_label
# End

fix how many number of each classs sample need to be in training and validation sets
negaive number represent all the data available in that case it will skip validation 
data and it comes out to be empty 

In [21]:
TRAIN_SIZE_PER_LABEL = 30000 # take TRAIN_SIZE_PER_LABEL data for training
VALIDATION_SIZE_PER_LABEL = 20000 # take VALIDATION_SIZE_PER_LABEL amount for validation 
TEST_SIZE = -1 # Negative Number represent All Data will be taken 

get merged training and validation set

In [22]:
training_set, training_label, validation_set, validation_label = merge_dataset(data_train=train_dict,
                                                                               size=TRAIN_SIZE_PER_LABEL,
                                                                               label_map=CLASS_MAP,
                                                                               validation_size=VALIDATION_SIZE_PER_LABEL)

get test set

In [27]:
test_set, test_label, _, _ = merge_dataset(data_train=test_dict, size=TEST_SIZE, label_map=CLASS_MAP, validation_size=None)

get random shuffled training data

In [24]:
training_set, training_label = randomize(training_set, training_label)

get random shuffled validation data

In [25]:
validation_set, validation_label = randomize(validation_set, validation_label)

get random shuffled test data not required

In [29]:
#test_set, test_label = randomize(test_set, test_label)

In [30]:
print('training data shape {0} training label shape:- {1}'.format(training_set.shape, training_label.shape))
print('validation data shape {0} validation label shape:- {1}'.format(validation_set.shape, validation_label.shape))
print('test data shape {0} test label shape:- {1}'.format(test_set.shape, test_label.shape))

training data shape (300000, 28, 28) training label shape:- (300000, 10)
validation data shape (200000, 28, 28) validation label shape:- (200000, 10)
test data shape (18724, 28, 28) test label shape:- (18724, 10)


In [33]:
#Random check from training Data check If valid random sampling occured or not
def random_check(dataset: np.array, label: np.array, index: int):
    plt.imshow(dataset[index].astype('uint8'), cmap=plt.get_cmap('gray'))
    print('label is ', CLASS_INV_MAP[np.argmax(label[index])])
# End

In [38]:
#random_check(training_set, training_label, 23 )
#random_check(validation_set, validation_label, 116)
#random_check(test_set, test_label, 1870)

In [39]:
def pickle_datasets(data: Dict[str, np.array], filepath: str, force=False)->None:
    """
    pickle all the important data
    
    parameter
    ---------
        data: Dict[str, np.array]
            data dictionary data name to data mapping
        
        filepath: str
            file path where to pickle the data
        
        force: bool (optional)
            if set to true then override the already pickled file
    
    return
    -------
        None
            
    """
    if os.path.exists(filepath) and not force:
        print('dataset already present skipping')
    print('writing dataset to {0} file..'.format(filepath))
    with open(filepath, 'wb') as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
# End

In [55]:
test_suffix = 'all' if TEST_SIZE < 0 else TEST_SIZE

In [56]:
dataset = {
    'trainingSet'+str(TRAIN_SIZE_PER_LABEL): training_set,
    'trainingLabel'+str(TRAIN_SIZE_PER_LABEL): training_label,
    'validationSet'+str(VALIDATION_SIZE_PER_LABEL): validation_set,
    'validationLabel'+str(VALIDATION_SIZE_PER_LABEL): validation_label,
    'testSet'+str(test_suffix): test_set,
    'testLabel'+str(test_suffix): test_label,
    'classLabelEncodingMapping': CLASS_MAP
}

In [57]:

ds_pickle_filepath = 'tr_set_{0}_val_{1}_test_{2}.pickle'.format(TRAIN_SIZE_PER_LABEL,
                                                                 VALIDATION_SIZE_PER_LABEL,
                                                                test_suffix)

In [54]:
pickle_datasets(dataset,ds_pickle_filepath, True)

writing dataset to tr_set_30000_val_20000_test_all.pickle file..


not in use just to create sample data to check for merge_dataset function
skip it, not in use in any ways, i am keeping this to test in case something
was not right

In [63]:
def test_data_gen(N, size, label_list):
    
    get_d = lambda l, n : [{'data': np.random.randint(1, 100, size), 'label': l} for i in range(n)]
    dat = {l:get_d(l, N) for l in label_list}
    return dat
# End


In [64]:
sample_data = test_data_gen(5, (2, 3), CLASS_LIST)
tr, lbt, val_d, val_lb = merge_dataset(sample_data, 4, CLASS_MAP, 1)

In [65]:
r_val, r_val_lab = randomize(val_d, val_lb)

In [66]:
print(tr.shape, lbt.shape, val_d.shape, val_lb.shape)

(40, 2, 3) (40, 10) (10, 2, 3) (10, 10)
